<a href="https://colab.research.google.com/github/comojin1994/GoogleNet/blob/master/Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
%cd ./drive/My\ Drive/Deep_learning/Paper_implementation/GoogleNet
!ls

/content/drive/My Drive/Deep_learning/Paper_implementation/GoogleNet
checkpoints  logs  model.py  __pycache__  Training.ipynb


In [3]:
%tensorflow_version 2.x
import tensorflow as tf
import numpy as np
import os, math
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
from tensorflow.keras import layers, datasets
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from datetime import datetime
from model import googleNet
tf.__version__

TensorFlow 2.x selected.


'2.1.0'

### Hyperparameter

In [0]:
EPOCHS = 1
batch_size = 32
initial_epoch = 0
num_classes = 10
learning_rate = 0.001
input_shape = (32, 32, 3)

loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate)
metrcis = ['accuracy']

### Callbacks

In [0]:
# Tensorboard
logdir = os.path.join('logs', datetime.now().strftime('%Y%m%d-%H%M%S'))
tensorboard = tf.keras.callbacks.TensorBoard(
      logdir,
      histogram_freq=1,
      write_graph=True,
      write_images=True
)

# Learning rate sheduler
def scheduler(epoch):
  if epoch < 10: return 0.001
  else: return 0.001 * math.exp(0.1 * (10 - epoch))

learning_rate_scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler, verbose=1)

# checkpoint
ckpt_dir = 'checkpoints'
checkpoint = tf.keras.callbacks.ModelCheckpoint(
      ckpt_dir,
      monitor='val_accuracy',
      verbose=1,
      save_best_only=True
)

callbacks = [tensorboard, learning_rate_scheduler, checkpoint]

### Load Data

In [6]:
(x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()

train_datagen = ImageDataGenerator(
    rescale=1./255.,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    dtype=tf.float32
)
validation_datagen = ImageDataGenerator(
    rescale=1./255.,
    dtype=tf.float32
)

train_generator = train_datagen.flow(x_train, y_train, batch_size=batch_size)
validation_generator = validation_datagen.flow(x_test, y_test, batch_size=batch_size)

170500096/170498071 [==============================] - 2s 0us/step


### Tensorboard

In [7]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

--2020-03-16 13:03:43--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 3.227.142.238, 52.4.250.149, 52.72.103.164, ...
Connecting to bin.equinox.io (bin.equinox.io)|3.227.142.238|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.13M  57.4MB/s    in 0.2s    

2020-03-16 13:03:43 (57.4 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [8]:
get_ipython().system_raw(
      'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
      .format(logdir))
get_ipython().system_raw('./ngrok http 6006 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c \
      "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

http://23e89340.ngrok.io


### Load and compile Model

In [9]:
model = googleNet(input_shape)
model.compile(optimizer=optimizer, loss=loss_object, metrics=metrcis,
              loss_weights={'out1':1., 'out2':0.3, 'out3':0.3})
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 32, 32, 64)   1792        input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 16, 16, 64)   0           conv2d[0][0]                     
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 16, 16, 64)   256         max_pooling2d[0][0]              
______________________________________________________________________________________________

### Training

In [10]:
history = model.fit(
      train_generator,
      epochs = EPOCHS,
      callbacks=callbacks,
      validation_data=validation_generator,
      initial_epoch=initial_epoch,
      steps_per_epoch=len(train_generator),
      validation_steps=len(validation_generator)
)

ValueError: ignored

### History

In [12]:
pred = model.predict(x_test[0:1].astype(np.float32))
pred = tf.argmax(pred, axis=2)
pred

<tf.Tensor: shape=(3, 1), dtype=int64, numpy=
array([[3],
       [7],
       [4]])>

In [10]:
x_test[0:1].shape

(1, 32, 32, 3)